In [36]:
import numpy as np
import pandas as pd
import mne
from mne_bids import (BIDSPath, read_raw_bids, find_matching_paths)
import xarray as xr
from ptsa.data.timeseries import TimeSeries
from ptsa.data.concat import concat
import os
# import cmldask.CMLDask as da
# import cmlreaders as cml
import pickle

def mne_to_ptsa(ep):
    '''Create an PTSA TimeSeries (essentially xarray) version of MNE epoch data'''
    assert ep.metadata is not None, "Please define mne.Epoch.metadata"
    ts = TimeSeries(
        ep.get_data(copy=True), dims=('event','channel','time'),
        coords={'event':pd.MultiIndex.from_frame(ep.metadata),
                'channel':ep.info['ch_names'],
                'time':ep.times,
                'samplerate':ep.info['sfreq']},
        name='data'
    )
    ts = ts.reset_index('event')
    return ts

In [34]:
ep = epochs
assert ep.metadata is not None, "Please define mne.Epoch.metadata"
ts = TimeSeries(
    ep.get_data(copy=True), dims=('event','channel','time'),
    coords={'event':pd.MultiIndex.from_frame(ep.metadata),
                        'channel':ep.info['ch_names'],
                        'time':ep.times,
                        'samplerate':ep.info['sfreq']},
    name='data'
    )
ts = ts.reset_index('event')
# ts = ts.set_index({"event": [coord for coord in ts.event.coords if coord != "samplerate"]})

In [5]:
subject = 'LTP453'
settings_path = "settings/phase1_settings.pkl"
settings = pickle.load(open(settings_path, 'rb'))
# data = cml.get_data_index(kind = 'ltp')
# data = data[(data['experiment']==settings['experiment'])&(data['subject']==subject)].sort_values('session').reset_index()

In [37]:
bids_root = "~/data/nicls_bids"
bids_root = os.path.expanduser(bids_root)
bids_paths = find_matching_paths(
    bids_root, 
    subjects=subject, 
    tasks="NiclsCourierReadOnly",
    datatypes="eeg",
    extensions=".bdf"
    )
all_sessions = []
for bids_path in bids_paths[0:1]:
    events_path = bids_path.copy().update(suffix="events", extension=".tsv")
    event_df = pd.read_csv(events_path, sep="\t")
    raw = read_raw_bids(bids_path, extra_params={"infer_types":True}, verbose=False)
    mne_events, mne_event_id = mne.events_from_annotations(raw, verbose=False)
    epochs = mne.Epochs(
        raw,
        mne_events, 
        event_id={'WORD':mne_event_id['WORD']}, 
        event_repeated='drop', 
        tmin=(settings['rel_start']-settings['buffer_time'])/1000.,
        tmax=(settings['rel_stop']+settings['buffer_time'])/1000., 
        baseline=None, 
        preload=True
    )
    epochs.metadata = event_df.query("trial_type=='WORD'").reset_index(drop=True)
    # do more stuff here
    all_sessions.append(mne_to_ptsa(epochs))
all_sessions = concat(all_sessions, 'event')
    

Not setting metadata
120 matching events found
No baseline correction applied
0 projection items activated
Loading data for 120 events and 4097 original time points ...


/var/folders/7h/662tdm8d6sn0krrht717wzmm0000gq/T/ipykernel_4676/2285922562.py:14: RuntimeWarning: The unit for channel(s) EXG5, EXG6, EXG7, EXG8 has changed from V to NA.
  raw = read_raw_bids(bids_path, extra_params={"infer_types":True}, verbose=False)
/var/folders/7h/662tdm8d6sn0krrht717wzmm0000gq/T/ipykernel_4676/2285922562.py:14: RuntimeWarning: The unit for channel(s) Status has changed from NA to V.
  raw = read_raw_bids(bids_path, extra_params={"infer_types":True}, verbose=False)
/var/folders/7h/662tdm8d6sn0krrht717wzmm0000gq/T/ipykernel_4676/2285922562.py:14: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status']
  raw = read_raw_bids(bids_path, extra_params={"infer_types":True}, verbose=False)
/var/folders/7h/662tdm8d6sn0krrht717wzmm0000gq/T/ipykernel_4676/2285922562.py:14: RuntimeWarning: Not setting positions of 9 eog/misc/stim channels found in montage:
['EXG1', 'EXG2'

0 bad epochs dropped
Adding metadata with 15 columns


In [46]:
from ptsa.data.filters import MorletWaveletFilter, ButterworthFilter

# powers = MorletWaveletFilter(freqs=settings['freqs'], output='power').filter(ts)

In [47]:
subject = "LTP453"
bids_root = "~/data/nicls_bids"
bids_root = os.path.expanduser(bids_root)
bids_paths = find_matching_paths(
    bids_root,
    subjects=subject,
    tasks="NiclsCourierReadOnly",
    datatypes="eeg",
    extensions=".bdf",
)
# read in data and compute features one session at a time
feats = []
for i, bids_path in enumerate(bids_paths):
    print(f"Reading session {i} data")
    # intialize data reader, load words events and buffered eeg epochs
    events_path = bids_path.copy().update(suffix="events", extension=".tsv")
    event_df = pd.read_csv(events_path, sep="\t")
    raw = read_raw_bids(
        bids_path, extra_params={"infer_types": True}, verbose=False
    )
    mne_events, mne_event_id = mne.events_from_annotations(raw, verbose=False)
    eeg = mne.Epochs(
        raw,
        mne_events,
        event_id={"WORD": mne_event_id["WORD"]},
        event_repeated="drop",
        tmin=(settings["rel_start"] - settings["buffer_time"]) / 1000.0,
        tmax=(settings["rel_stop"] + settings["buffer_time"]) / 1000.0,
        baseline=None,
        preload=True,
    )
    eeg.metadata = event_df.query("trial_type=='WORD'").reset_index(drop=True)
    eeg = mne_to_ptsa(eeg)
    # select relevant channels
    if settings["reference"] == "average":
        eeg = eeg[:, :128]
        if (
            eeg.channel[0].str.startswith("E") and not settings["clean"]
        ):  # EGI system
            eeg.drop_sel({"channel": ["E8", "E25", "E126", "E127"]})
        eeg -= eeg.mean("channel")
    elif settings["reference"] == "bipolar":
        bipolar_pairs = np.loadtxt(
            "/home1/jrudoler/biosemi_cap_bipolar_pairs.txt", dtype=str
        )
        mapper = MonopolarToBipolarMapper(bipolar_pairs, channels_dim="channel")
        eeg = eeg.filter_with(mapper)
        eeg = eeg.assign_coords(
            {"channel": np.array(["-".join(pair) for pair in eeg.channel.values])}
        )
    else:
        raise ValueError("reference setting unknown")
    # filter out line noise at 60 and 120Hz
    eeg = ButterworthFilter(filt_type="stop", freq_range=[58, 62], order=4).filter(
        eeg
    )
    eeg = ButterworthFilter(
        filt_type="stop", freq_range=[118, 122], order=4
    ).filter(eeg)
    # highpass filter to account for drift
    eeg = ButterworthFilter(filt_type="highpass", freq_range=1).filter(eeg)
    pows = MorletWaveletFilter(
        settings["freqs"], width=settings["width"], output="power", cpus=25
    ).filter(eeg)
    del eeg
    pows = (
        pows.remove_buffer(settings["buffer_time"] / 1000)
        + np.finfo(float).eps / 2.0
    )
    pows = pows.reduce(np.log10)
    # swap order of events and frequencies --> result is events x frequencies x channels x time
    # next, average over time
    pows = pows.transpose("event", "frequency", "channel", "time").mean("time")
    # reshape as events x features
    pows = pows.stack(features=("frequency", "channel"))
    # normalize along event axis
    if normalize:
        pows = pows.reduce(func=zscore, dim="event", keep_attrs=True, ddof=1)
    feats.append(pows)
    del pows
feats = concat(feats, dim="event")

Reading session 0 data
Not setting metadata
120 matching events found
No baseline correction applied
0 projection items activated
Loading data for 120 events and 4097 original time points ...


/var/folders/7h/662tdm8d6sn0krrht717wzmm0000gq/T/ipykernel_4676/1537456261.py:18: RuntimeWarning: The unit for channel(s) EXG5, EXG6, EXG7, EXG8 has changed from V to NA.
  raw = read_raw_bids(
/var/folders/7h/662tdm8d6sn0krrht717wzmm0000gq/T/ipykernel_4676/1537456261.py:18: RuntimeWarning: The unit for channel(s) Status has changed from NA to V.
  raw = read_raw_bids(
/var/folders/7h/662tdm8d6sn0krrht717wzmm0000gq/T/ipykernel_4676/1537456261.py:18: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status']
  raw = read_raw_bids(
/var/folders/7h/662tdm8d6sn0krrht717wzmm0000gq/T/ipykernel_4676/1537456261.py:18: RuntimeWarning: Not setting positions of 9 eog/misc/stim channels found in montage:
['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set

0 bad epochs dropped
Adding metadata with 15 columns
CPP total time wavelet loop:  7.738828897476196


NameError: name 'normalize' is not defined

: 

Check against cmlreaders

In [ ]:
import cmlreaders as cml
data = cml.get_data_index(kind = 'ltp')
data = data[(data['experiment']==settings['experiment'])&(data['subject']==subject)].sort_values('session').reset_index()
# cml.CMLReader(subject=subject, experiment=row['experiment'], session=row['session'])
r = cml.CMLReader(subject, experiment=settings['experiment'], session=0)
evs = r.load('task_events')
word_evs = evs[(evs.type=='WORD')&(evs.eegoffset!=-1)]
if len(word_evs)==0:
    # continue # sync pulses not recorded
    print("bad!")
eeg = r.load_eeg(word_evs,
                    rel_start=settings['rel_start'] - settings['buffer_time'],
                    rel_stop=settings['rel_stop'] + settings['buffer_time'],
                    clean=settings['clean']
                ).to_ptsa()

: 

In [ ]:
assert (eeg.item == all_sessions[0].item).all()
assert (all_sessions[0]['item'].values == eeg['item'].values).all()

: 

: 